In [1]:
import time
from os import walk
import os.path as ospa
import numpy as np
import re
import matplotlib.pyplot as plt
# import mpl_toolkits.mplot3d.axes3d as p3
# import matplotlib.animation as animation
import pandas as pd

%matplotlib notebook 
%matplotlib notebook 

plt.rcParams.update({'font.size': 14})

# dataPath = ".\Pilot20220325LeakOil"
# dataPath = ".\CurrentData"
dataPath = ".\TMPData"

In [2]:
'''
General function
'''

def aPlot(figName, is3D = False):
    ax = []
    
    fig1 = plt.figure(figsize = (12,4))
    fig1.suptitle(figName, fontsize=16)
    if(is3D):
        ax = fig1.add_subplot(111, projection='3d')
    else:
        ax = fig1.add_subplot(111)
        
    return ax

# The default decoding format is one float number
def decodeTrackData(dataFormat, dataStr, numFormat = "\d+\.\d+", numNum = 1):
    findResult = re.findall(dataFormat, aLine)
    
    findData = []
    
    if findResult:
        findNums = re.findall(numFormat, findResult[0])
        if(len(findNums) != numNum):
            print("Incorrectly find %d %s given %d in command" % (len(findNums), numFormat, numNum))
            return findData
        else:
            for num in findNums:
                try:
                    num = float(num)
                except:
                    pass               
                findData.append(num)

    else: # Padding None if no number is found
        findData = [None] * numNum
        
    return findData

def decodeTaskSummary(taskFormat, completeStatusFormat, completeTimeFormat, dataStr):
    findData = []
    
    taskSummaries = re.findall(taskFormat+completeStatusFormat+completeTimeFormat, dataStr)
    
    for taskSummary in taskSummaries:
        res = re.findall(taskFormat+"|"+completeStatusFormat+"|"+completeTimeFormat, taskSummary)
        try:
            res[2] = float(res[2])
        except:
            pass   
        findData.append(res)
    
    return findData
    

In [3]:
'''
Data format
'''

vrDataInfo = [] # dataStamp, trialNum, voltageLevel, startTime, score
vrDataTrack = {} # time (secs), indexX, indexY, indexZ, thumbX, thumbY, thumbZ, indexDist, thumbDist, object
vrDataSummary = {} # object, taskResult, competeTime (secs)

for root, directories, files in walk(dataPath):
    for fileName in files:
        print(fileName)
        if(re.search('\.txt', fileName)):
            condi = re.split('_|\.txt', fileName)
            dataStamp = condi[0]
            trialNum = int(re.findall("\d+", condi[-2])[0])

            file1 = open(ospa.join(root, fileName), 'r')
            fileLines = file1.readlines()
            
            dataTrack = [] # time (secs), indexX, indexY, indexZ, thumbX, thumbY, thumbZ, indexDist, thumbDist, object
            subjectId = None
            taskId = None
            voltageLevel = None # voltageLevel, 
            startTime = None # startTime (Universal Time)
            dataSummary = [] # object, taskResult, competeTime (secs)
            score = None # Final score               
            
            for aLine in fileLines:
                if(aLine[0] == 't'):  
                    aTrackFrame = []
                    
                    # time (secs)
                    aTrackFrame.extend(decodeTrackData("t\d+\.\d+", aLine))
                    
                    # (indexX, indexY, indexZ)
                    aTrackFrame.extend(decodeTrackData("le\(\d+\.\d+, \d+\.\d+, \d+\.\d+\)", aLine, "\d+\.\d+", 3))
            
                    # (thumbX, thumbY, thumbZ)
                    aTrackFrame.extend(decodeTrackData("re\(\d+\.\d+, \d+\.\d+, \d+\.\d+\)", aLine, "\d+\.\d+", 3))
                    
                    # (indexX, indexY, indexZ)
                    aTrackFrame.extend(decodeTrackData("in\(\d+\.\d+, \d+\.\d+, \d+\.\d+\)", aLine, "\d+\.\d+", 3))
            
                    # (thumbX, thumbY, thumbZ)
                    aTrackFrame.extend(decodeTrackData("th\(\d+\.\d+, \d+\.\d+, \d+\.\d+\)", aLine, "\d+\.\d+", 3))
                    
                    # indexDist
                    aTrackFrame.extend(decodeTrackData("indi\d+\.\d+", aLine))
                    
                    # thumbDist
                    aTrackFrame.extend(decodeTrackData("thdi\d+\.\d+", aLine))
                    
                    # object (Name)
                    aTrackFrame.extend(decodeTrackData("Grape\d+ | Key\d+", aLine, "Grape\d+ | Key\d+"))
             
                    dataTrack.append(aTrackFrame)
                
                elif(aLine[0] == 'v'):
                    voltageLevel = decodeTrackData("voltLevel\d+", aLine, "\d+")[0]
                    
                    startTime = decodeTrackData("starttimestamp\d+", aLine, "\d+")[0]
                    
                    subjectId = decodeTrackData("subjectid\d+", aLine, "\d+")[0]
                    
                    taskId = decodeTrackData("taskid\d+", aLine, "\d+")[0]
                    
                    completeInfo = re.findall("completeinfo.*", aLine)[0]
                    
                    dataSummary = []
                    dataSummary.extend(decodeTaskSummary("Grape\d+", "[a-zA-Z]+", "\d+\.\d+", completeInfo))
                    dataSummary.extend(decodeTaskSummary("A\d+E\d+", "[a-zA-Z]+", "\d+\.\d+", completeInfo))
                    
                    score = decodeTrackData("score\d+", completeInfo, "\d+")[0]
            
            print(dataStamp)
            
            vrDataTrack[dataStamp] = pd.DataFrame(dataTrack, columns = ['Time', 'LeftEyeX', 'LeftEyeY', 'LeftEyeZ', 
                                                                        'RightEyeX', 'RightEyeY', 'RightEyeZ', 
                                                                        'IndexX', 'IndexY', 'IndexZ', 
                                                                        'ThumbX', 'ThumbY', 'ThumbZ', 
                                                                        'IndexDist', 'ThumbDist', 'Object'])
            
            vrDataSummary[dataStamp] = pd.DataFrame(dataSummary, columns = ['Object', 'TaskResult', 'CompeteTime'])
            
            vrDataInfo.append([dataStamp, trialNum, subjectId, taskId, voltageLevel, startTime, score])
            
vrDataInfo = pd.DataFrame(vrDataInfo, columns = ['Label', 'Trial', 'SubjectId', 'TaskId', 'VoltLevel', 
                                                 'StartTime', 'Score'])        
print(vrDataInfo)

Data13-50-47_VR_t00.txt
Data13-50-47
Data14-13-43_VR_t01.txt
Data14-13-43
Data14-20-49_VR_t02.txt
Data14-20-49
Data14-22-58_VR_t03.txt
Data14-22-58
          Label  Trial  SubjectId  TaskId  VoltLevel  StartTime  Score
0  Data13-50-47      0        0.0     0.0      100.0      132.0    0.0
1  Data14-13-43      1        0.0     0.0      100.0      552.0    0.0
2  Data14-20-49      2        0.0     0.0      100.0       47.0    2.0
3  Data14-22-58      3        0.0     0.0      100.0      189.0    0.0


In [ ]:
selectLabel = dataStamp # The most recent data
selectLabel = 'Data21-17-54'

trData = vrDataTrack[selectLabel]
trData.head()

In [ ]:
'''
Plot Tracking data
'''
t = trData['Time'].values
t = t - t[0]

eyeCenter = np.array([0.5 * (trData['LeftEyeX'].values + trData['RightEyeX'].values), 
                      0.5 * (trData['LeftEyeY'].values + trData['RightEyeY'].values), 
                      0.5 * (trData['LeftEyeZ'].values + trData['RightEyeZ'].values)]).T

eyeLinkVect = np.array([trData['LeftEyeX'].values - trData['RightEyeX'].values, 
               trData['LeftEyeY'].values - trData['RightEyeY'].values, 
               trData['LeftEyeZ'].values - trData['RightEyeZ'].values]).T

index = np.array([trData['IndexX'].values, trData['IndexY'].values, trData['IndexZ'].values]).T
thumb = np.array([trData['ThumbX'].values, trData['ThumbY'].values, trData['ThumbZ'].values]).T

fig1 = plt.figure()
ax = fig1.add_subplot(111, projection='3d')
ax.set_xlabel('X (m)')
ax.set_ylabel('Z (m)')
ax.set_zlabel('Y (m)')

ax.set_xlim([1.2, 2.0])
ax.set_ylim([0.0, 2.0])
ax.set_zlim([1.0, 2.0])

axHandle = ax.plot([0, 0]+eyeCenter[0,0], [0, 0]+eyeCenter[0,2], [0, 1]+eyeCenter[0,1],'tab:green')[0]
axHandle2 = ax.plot(eyeCenter[0,0], eyeCenter[0,2], eyeCenter[0,1],'or')[0]

axHandle3 = ax.plot(index[0,0], index[0,2], index[0,1],'ob')[0]
axHandle4 = ax.plot(thumb[0,0], thumb[0,2], thumb[0,1],'oc')[0]

ax.view_init(elev = 45, azim=0.0)

for t_i, eC, eL, index_i, thumb_i in zip(t, eyeCenter, eyeLinkVect, index, thumb):
    plt.title("Time = %.3f sec" % t_i)
    
    normVect = np.cross([0,1,0], eL)  
    normVect = normVect / np.linalg.norm(normVect)
    
    axHandle.set_xdata([0, normVect[0]]+eC[0])
    axHandle.set_ydata([0, normVect[2]]+eC[2])
    axHandle.set_3d_properties([0, normVect[1]]+eC[1])
    
    axHandle2.set_xdata(eC[0])
    axHandle2.set_ydata(eC[2])
    axHandle2.set_3d_properties(eC[1])
    
    axHandle3.set_xdata(index_i[0])
    axHandle3.set_ydata(index_i[2])
    axHandle3.set_3d_properties(index_i[1])
    
    axHandle4.set_xdata(thumb_i[0])
    axHandle4.set_ydata(thumb_i[2])
    axHandle4.set_3d_properties(thumb_i[1])
    
    fig1.canvas.draw()